In [1]:
# import os, re, numpy as np, pandas as pd
# import statsmodels.formula.api as smf
#
# from scipy.stats import norm
#
# SRC_XLSX = r"G:\25国赛\C题\PythonProject\code3\工作簿4.xlsx"
# OUT_DIR  = r"G:\25国赛\C题\PythonProject\code3"
#
# # --- helper functions ---
# def pick_col(df: pd.DataFrame, candidates):
#     for name in candidates:
#         if name in df.columns:
#             return name
#     low = {c.lower(): c for c in df.columns}
#     for name in candidates:
#         k = re.sub(r"\s+", "", name.lower())
#         for c in low:
#             if re.sub(r"\s+", "", c) == k:
#                 return low[c]
#     raise KeyError(f"Missing column. Need one of {candidates}. Actual: {list(df.columns)}")
#
# def to_weeks(series: pd.Series) -> pd.Series:
#     s = pd.to_numeric(series, errors="coerce")
#     return s/7.0 if s.median() > 40 else s
#
# # --- load data ---
# xls = pd.ExcelFile(SRC_XLSX)
# df = pd.read_excel(SRC_XLSX, sheet_name=0).copy()
#
# # column aliases
# COL_ALIASES = {
#     "age":   ["年龄", "孕妇年龄", "Age"],
#     "ga":    ["检测孕期", "孕周(天)", "孕周(周)", "孕周", "孕周天", "孕周(天数)", "检测孕周", "GA", "gestational_age"],
#     "bmi":   ["孕妇BMI", "BMI", "体质指数"],
#     "reads": ["原始读段数", "读段数", "原始reads", "reads"],
#     "yfrac": ["Y染色体浓度", "Y浓度", "Y_fraction", "Y%", "yfrac"],
#     "grav":  ["怀孕次数", "孕次", "gravidity", "怀孕史"]
# }
#
# # map columns
# col_map = {}
# for std, cand in COL_ALIASES.items():
#     try:
#         col_map[std] = pick_col(df, cand)
#     except KeyError:
#         if std == "grav":
#             col_map[std] = None
#         else:
#             raise
#
# rename_dict = {col_map[k]: k for k in col_map if col_map[k] is not None}
# df = df.rename(columns=rename_dict)
#
# # preprocess
# if col_map["grav"] is None:
#     df["grav"] = np.nan
# df["ga_weeks"] = to_weeks(df["ga"])
# df["log_reads"] = np.log1p(pd.to_numeric(df["reads"], errors="coerce"))
# df["age"]  = pd.to_numeric(df["age"], errors="coerce")
# df["bmi"]  = pd.to_numeric(df["bmi"], errors="coerce")
# df["yfrac"]= pd.to_numeric(df["yfrac"], errors="coerce")
#
# grav = df["grav"].astype(str).str.replace("≥", ">=").str.strip()
# grav = grav.replace({">=3":"ge3", "3":"ge3", "1":"g1", "2":"g2"})
# grav = np.where(grav.isin(["g1","g2","ge3"]), grav, "unk")
# df["grav_cat"] = grav
#
# df = df.dropna(subset=["ga_weeks","age","bmi","log_reads","yfrac"]).reset_index(drop=True)
# df = df[(df["ga_weeks"]>0) & (df["ga_weeks"]<43)].reset_index(drop=True)
#
# # BMI quintiles
# labels = [f"Q{i}" for i in range(1,6)]
# df["bmi_group"], bins = pd.qcut(df["bmi"], q=5, labels=labels, retbins=True, duplicates="drop")
#
# # model
# W_MIN, W_MAX = 12.0, 25.0
# STEP = 0.1
# THRESH = 0.040
# grid_min = float(max(W_MIN, df["ga_weeks"].min()))
# grid_max = float(min(W_MAX, df["ga_weeks"].max()))
# week_grid = np.round(np.arange(grid_min, grid_max + 1e-9, STEP), 2)
#
# formula = "yfrac ~ bs(ga_weeks, df=4) + age + bmi + log_reads + C(grav_cat) + C(bmi_group)"
# ols = smf.ols(formula, data=df).fit()
# sigma = ols.resid.std(ddof=int(ols.df_model + 1))
#
# # predict mu and convert to probability
# def predict_mu_by_week(weeks):
#     mu = []
#     for w in weeks:
#         newdf = pd.DataFrame({
#             "ga_weeks": np.repeat(w, len(df)),
#             "age": df["age"].values,
#             "bmi": df["bmi"].values,
#             "log_reads": df["log_reads"].values,
#             "grav_cat": df["grav_cat"].values,
#             "bmi_group": df["bmi_group"].values
#         })
#         mu.append(ols.predict(newdf).values)
#     return np.vstack(mu)
#
# MU = predict_mu_by_week(week_grid)
# P  = 1.0 - norm.cdf((THRESH - MU)/sigma)
#
# # risk minimization per BMI group
# C_FAIL, C_DELAY = 1.0, 0.04
# rows = []
# for lab in labels:
#     idx = (df["bmi_group"]==lab).values
#     if idx.sum()==0:
#         continue
#     P_group = P[:, idx].mean(axis=1)
#     risk = C_FAIL*(1.0 - P_group) + C_DELAY*np.maximum(0.0, week_grid - W_MIN)
#     k = int(np.argmin(risk))
#     rows.append({
#         "BMI组": lab,
#         "人数": int(idx.sum()),
#         "BMI范围": f"[{df.loc[idx,'bmi'].min():.3f}, {df.loc[idx,'bmi'].max():.3f}]",
#         "推荐最优周": float(week_grid[k]),
#         "该周达标概率(组均值)": float(P_group[k]),
#         "最小风险": float(risk[k])
#     })
#
# result = pd.DataFrame(rows).sort_values("推荐最优周").reset_index(drop=True)
#
# # export
# os.makedirs(OUT_DIR, exist_ok=True)
# out_csv = os.path.join(OUT_DIR, "第三问_B1_最优周_基线.csv")
# result.to_csv(out_csv, index=False)
# ols_path = os.path.join(OUT_DIR, "第三问_B1_OLS模型_摘要.txt")
# with open(ols_path, "w", encoding="utf-8") as f:
#     f.write(ols.summary().as_text())
#
# result, out_csv, ols_path


(  BMI组   人数             BMI范围  推荐最优周  该周达标概率(组均值)      最小风险
 0   Q1  206  [20.703, 29.984]   12.5     0.890407  0.129593
 1   Q2  205  [29.996, 31.209]   12.6     0.871841  0.152159
 2   Q3  205  [31.210, 32.691]   12.7     0.864213  0.163787
 3   Q4  206  [32.701, 34.450]   12.7     0.861707  0.166293
 4   Q5  205  [34.466, 46.875]   13.0     0.757273  0.282727,
 'G:\\25国赛\\C题\\PythonProject\\code3\\第三问_B1_最优周_基线.csv',
 'G:\\25国赛\\C题\\PythonProject\\code3\\第三问_B1_OLS模型_摘要.txt')

In [2]:
# Re-run pipeline with qualified threshold = 0.60 (记录级)

import pandas as pd
import numpy as np
import os

PATH2 = r"G:\25国赛\C题\PythonProject\code3\工作簿2_筛选结果(1).xlsx"
PATH4 = r"G:\25国赛\C题\PythonProject\code3\工作簿4.xlsx"
OUT_DIR = ""

THR = 0.040
W_MIN, W_MAX, STEP = 10.0, 25.0, 0.1
C_FAIL, C_DELAY = 1.0, 0.04
QUAL_THRESHOLD = 0.60  # 修改为60%

def to_weeks(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    return s/7.0 if s.median() > 40 else s

# 工作簿2
df2 = pd.read_excel(PATH2, sheet_name=0).copy()
df2 = df2.rename(columns={
    "年龄":"age","检测孕期":"ga","孕妇BMI":"bmi",
    "原始读段数":"reads","Y染色体浓度":"yfrac","怀孕次数":"grav"
})
df2["ga_weeks"] = to_weeks(df2["ga"])
df2 = df2.dropna(subset=["bmi","ga_weeks"]).reset_index(drop=True)

labels = [f"Q{i}" for i in range(1,6)]
df2["bmi_group"], bmi_bins = pd.qcut(df2["bmi"], q=5, labels=labels, retbins=True, duplicates="drop")

week_grid = np.round(np.arange(W_MIN, W_MAX+1e-9, STEP), 2)

def empirical_success_prob(weeks_array: np.ndarray, w: float) -> float:
    if len(weeks_array)==0: return np.nan
    return float(np.mean(weeks_array <= w))

def risk_curve_from_weeks(weeks_array: np.ndarray, week_grid: np.ndarray):
    F = np.array([empirical_success_prob(weeks_array, w) for w in week_grid])
    R = C_FAIL*(1.0 - F) + C_DELAY*np.maximum(0.0, week_grid - W_MIN)
    return F, R

w2_curves = {}
for lab in labels:
    wk = df2.loc[df2["bmi_group"]==lab, "ga_weeks"].to_numpy()
    F2, R2 = risk_curve_from_weeks(wk, week_grid)
    w2_curves[lab] = {"F2":F2,"R2":R2,"n2":len(wk)}

# 工作簿4
df4 = pd.read_excel(PATH4, sheet_name=0).copy()
df4 = df4.rename(columns={
    "孕妇代码":"pid","年龄":"age","检测孕期":"ga","孕妇BMI":"bmi",
    "原始读段数":"reads","Y染色体浓度":"yfrac","怀孕次数":"grav"
})
df4["ga_weeks"] = to_weeks(df4["ga"])
df4["yfrac"] = pd.to_numeric(df4["yfrac"], errors="coerce")
df4["bmi"]   = pd.to_numeric(df4["bmi"], errors="coerce")
edges = bmi_bins.copy()
edges[0] -= 1e-6; edges[-1]+=1e-6
df4["bmi_group"] = pd.cut(df4["bmi"], bins=edges, labels=labels, include_lowest=True)
df4v = df4.dropna(subset=["ga_weeks","yfrac","bmi_group"]).copy()

w4_curves = {}
for lab in labels:
    sub = df4v[df4v["bmi_group"]==lab]
    n_rec = len(sub)
    if n_rec==0:
        w4_curves[lab]={"P4":np.full_like(week_grid,np.nan,dtype=float),"n4":0}
        continue
    ga_arr=sub["ga_weeks"].to_numpy(); y_arr=sub["yfrac"].to_numpy()
    P4 = np.array([ np.mean((ga_arr<=w)&(y_arr>=THR)) for w in week_grid ],dtype=float)
    w4_curves[lab]={"P4":P4,"n4":n_rec}

rows_final=[]
for lab in labels:
    F2=w2_curves[lab]["F2"]; R2=w2_curves[lab]["R2"]; P4=w4_curves[lab]["P4"]
    n2=w2_curves[lab]["n2"]; n4=w4_curves[lab]["n4"]
    ok=(P4>=QUAL_THRESHOLD)
    if not np.any(ok):
        rows_final.append({"BMI组":lab,"工作簿2_样本数":n2,"工作簿4_记录数":n4,
                           "合格阈值(记录级)":QUAL_THRESHOLD,"是否有合格周":"否",
                           "合格最优周":np.nan,"合格最优周_R2(来自W2)":np.nan,
                           "合格最优周_P4(来自W4_记录级)":np.nan})
        continue
    R2_masked=np.where(ok,R2,np.inf)
    k=int(np.argmin(R2_masked))
    rows_final.append({"BMI组":lab,"工作簿2_样本数":n2,"工作簿4_记录数":n4,
                       "合格阈值(记录级)":QUAL_THRESHOLD,"是否有合格周":"是",
                       "合格最优周":float(week_grid[k]),
                       "合格最优周_R2(来自W2)":float(R2[k]),
                       "合格最优周_P4(来自W4_记录级)":float(P4[k])})

final_table=pd.DataFrame(rows_final).sort_values("BMI组").reset_index(drop=True)
out_final="第三问_step3_合格且风险最低_推荐周_按记录计数_阈值60.csv"
final_table.to_csv(out_final,index=False)
final_table, out_final


(  BMI组  工作簿2_样本数  工作簿4_记录数  合格阈值(记录级) 是否有合格周  合格最优周  合格最优周_R2(来自W2)  \
 0   Q1        52       144        0.6      是   16.8        0.329692   
 1   Q2        53       165        0.6      是   19.0        0.435472   
 2   Q3        50       259        0.6      是   20.3        0.412000   
 3   Q4        52       204        0.6      是   16.3        0.290462   
 4   Q5        52       253        0.6      是   23.5        0.578462   
 
    合格最优周_P4(来自W4_记录级)  
 0            0.611111  
 1            0.606061  
 2            0.675676  
 3            0.602941  
 4            0.640316  ,
 '第三问_step3_合格且风险最低_推荐周_按记录计数_阈值60.csv')